In [3]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [4]:
%load_ext sql
%sql sqlite:///store.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Задание 1
Вывести страну, где популярнее всего группа Iron Maiden (т.е. треков куплено больше, чем в других странах). Также вывести кол-во купленных треков.

In [6]:
%%sql
SELECT BillingCountry, sum(
(SELECT sum(Quantity) FROM invoice_items i WHERE i.InvoiceId = inv.InvoiceId AND TrackId IN
  (SELECT TrackId	FROM tracks WHERE AlbumId	IN
    (SELECT AlbumId	FROM albums WHERE ArtistId IN
      (SELECT ArtistId FROM artists WHERE Name ='Iron Maiden'))))) as quantity
FROM invoices inv WHERE InvoiceId IN
  (SELECT InvoiceId	FROM invoice_items WHERE TrackId IN
    (SELECT TrackId	FROM tracks WHERE AlbumId	IN
      (SELECT AlbumId	FROM albums WHERE ArtistId IN
        (SELECT ArtistId FROM artists WHERE Name ='Iron Maiden'))))
GROUP BY BillingCountry
ORDER BY quantity DESC
LIMIT 1

 * sqlite:///store.db
Done.


BillingCountry,quantity
USA,61


In [7]:
%%sql
SELECT inv.BillingCountry, SUM(ii.Quantity) as quantity
FROM invoices inv
JOIN invoice_items ii ON inv.InvoiceId = ii.InvoiceId
JOIN tracks t ON ii.TrackId = t.TrackId
JOIN albums a ON t.AlbumId = a.AlbumId
JOIN artists ar ON a.ArtistId = ar.ArtistId
  WHERE ar.Name = 'Iron Maiden'
GROUP BY inv.BillingCountry
ORDER BY quantity DESC
LIMIT 1

 * sqlite:///store.db
Done.


BillingCountry,quantity
USA,61


## Задание 2
Вывести "средний по длине" трек, среди приобретённых компаниями (только клиентами из конкретных компаний) после 22 апреля 2023 года.
"Средний по длине" = максимально близкий к среднему значению множества треков по длине (не медиана!). Если таких треков несколько, вывести первый по алфавиту.

In [8]:
%%sql
SELECT TrackId, Name, ABS(Milliseconds	- (SELECT AVG(Milliseconds) FROM tracks)) AS difference FROM tracks
ORDER BY difference ASC
LIMIT(10)

 * sqlite:///store.db
Done.


TrackId,Name,difference
806,Wicked Ways,91.78789608908119
1649,Ten Years Gone,483.2121039109188
3279,Fairies Wear Boots,835.2121039109188
3413,"Concerto for Clarinet in A Major, K. 622: II. Adagio",882.7878960890812
3415,Symphony No.5 in C Minor: I. Allegro con brio,1137.2121039109188
647,Pot-Pourri N.º 4,1162.2121039109188
818,Anya,1162.2121039109188
3028,Zooropa,1240.2121039109188
176,The Winner Loses,1345.2121039109188
3136,Looking For Love,1658.2121039109188


In [9]:
%%sql
SELECT t.TrackId, t.Name, ABS(t.Milliseconds	- (SELECT AVG(t.Milliseconds) FROM tracks t)) AS difference FROM tracks t
JOIN invoice_items inv ON t.TrackId = inv.TrackId
JOIN invoices i ON inv.InvoiceId	= i.InvoiceId
JOIN Customers c ON i.CustomerId = c.CustomerId
  WHERE c.Company IS NOT NULL AND  i.InvoiceDate	> '2023-04-22'
ORDER BY difference ASC, Name ASC
LIMIT(1)

 * sqlite:///store.db
Done.


TrackId,Name,difference
1649,Ten Years Gone,483.2121039109188


In [10]:
%%sql
SELECT t.TrackId, t.Name,
ABS(t.Milliseconds	- (SELECT AVG(t.Milliseconds) FROM tracks t
  JOIN invoice_items inv ON t.TrackId = inv.TrackId
  JOIN invoices i ON inv.InvoiceId	= i.InvoiceId
  JOIN Customers c ON i.CustomerId = c.CustomerId
    WHERE c.Company IS NOT NULL AND  i.InvoiceDate	> '2023-04-22')) AS difference
FROM tracks t
JOIN invoice_items inv ON t.TrackId = inv.TrackId
JOIN invoices i ON inv.InvoiceId	= i.InvoiceId
JOIN Customers c ON i.CustomerId = c.CustomerId
  WHERE c.Company IS NOT NULL AND  i.InvoiceDate	> '2023-04-22'
ORDER BY difference ASC, Name ASC
LIMIT(1)

 * sqlite:///store.db
Done.


TrackId,Name,difference
1601,No Quarter,1250.2002472187742


## Задание 3

Вывести треки, встречающиеся и в нескольких плейлистах, и в нескольких заказах одновременно.

In [11]:
%%sql
SELECT inv.TrackId FROM invoice_items inv
JOIN invoice_items ii ON ii.TrackId = inv.TrackId
  WHERE inv.InvoiceId IS NOT ii.InvoiceId
INTERSECT
SELECT pl.TrackId FROM playlist_track pl
JOIN playlist_track pt ON pt.TrackId = pl.TrackId
  WHERE pl.PlaylistId IS NOT pt.PlaylistId

 * sqlite:///store.db
Done.


TrackId
894
2832
2850
2868
3177
3200
3214
3223
3250


In [12]:
%%sql
SELECT t.TrackId, t.Name FROM tracks t
JOIN invoice_items inv ON t.TrackId = inv.TrackId
JOIN invoice_items ii ON ii.TrackId = inv.TrackId
  WHERE inv.InvoiceId IS NOT ii.InvoiceId
INTERSECT
SELECT tt.TrackId, tt.Name FROM tracks tt
JOIN playlist_track pl ON tt.TrackId = pl.TrackId
JOIN playlist_track pt ON pt.TrackId = pl.TrackId
  WHERE pl.PlaylistId IS NOT pt.PlaylistId

 * sqlite:///store.db
Done.


TrackId,Name
894,Sunshine Of Your Love
2832,The Woman King
2850,The Fix
2868,Walkabout
3177,Hot Girl
3200,Gay Witch Hunt
3214,Phyllis's Wedding
3223,How to Stop an Exploding Man
3250,Pilot


## Задание 4
Вывести альбом, который является максимальным по продолжительности среди альбомов со стоимостью, большей средней цены за альбом *(цена альбома = сумма цен треков, продолжительность считать аналогично)*.

In [13]:
%%sql
WITH aa AS
(
SELECT a.AlbumId, a.Title, SUM(t.UnitPrice) AS cost , SUM(t.Milliseconds) AS duration
FROM albums a
JOIN tracks t ON a.AlbumId = t.AlbumId
GROUP BY a.AlbumId
)
SELECT* FROM aa
GROUP BY aa.AlbumId
HAVING aa.cost>(SELECT AVG(aa.cost) FROM aa)
ORDER BY aa.duration DESC
LIMIT(1)

 * sqlite:///store.db
Done.


AlbumId,Title,cost,duration
229,"Lost, Season 3",50.74000000000001,70665582


## Задание 5
Для каждого исполнителя, у которого есть песни нескольких жанров, найти жанр, по которому он заработал наибольшее количество денег, а также вывести процентное соотношение суммы, заработанной исполнителем по этому жанру к общей сумме заработанных денег. Формат вывода: имя исполнителя, жанр, соотношение.

In [14]:
%%sql
SELECT ar.ArtistId, ar.Name, SUM(t.UnitPrice) AS profit
FROM artists ar
JOIN albums a ON ar.ArtistId = a.ArtistId
JOIN tracks t ON a.AlbumId	= t.AlbumId
JOIN genres g ON t.GenreId	= g.GenreId
GROUP BY ar.ArtistId, ar.Name
HAVING COUNT(distinct t.GenreId)>1

 * sqlite:///store.db
Done.


ArtistId,Name,profit
6,Antônio Carlos Jobim,51.690000000000005
8,Audioslave,55.10000000000002
21,Various Artists,90.93999999999994
27,Gilberto Gil,46.68000000000002
81,Eric Clapton,91.51999999999994
82,Faith No More,76.97999999999998
84,Foo Fighters,75.56
88,Guns N' Roses,55.580000000000034
90,Iron Maiden,306.3700000000009
92,Jamiroquai,43.18000000000002


In [15]:
%%sql
SELECT ar.ArtistId, ar.Name, g.Name AS genre, SUM(t.UnitPrice) AS pprofit
FROM artists ar
JOIN albums a ON ar.ArtistId = a.ArtistId
JOIN tracks t ON a.AlbumId = t.AlbumId
JOIN genres g ON t.GenreId = g.GenreId
GROUP BY ar.ArtistId, ar.Name, g.Name

 * sqlite:///store.db
Done.


ArtistId,Name,genre,pprofit
1,AC/DC,Rock,33.31999999999999
2,Accept,Rock,7.960000000000001
3,Aerosmith,Rock,31.35
4,Alanis Morissette,Rock,18.869999999999997
5,Alice In Chains,Rock,18.88
6,Antônio Carlos Jobim,Jazz,21.859999999999996
6,Antônio Carlos Jobim,Latin,29.82999999999999
7,Apocalyptica,Metal,14.92
8,Audioslave,Alternative,13.860000000000001
8,Audioslave,Alternative & Punk,17.88


In [16]:
%%sql
WITH p AS
(
  SELECT ar.ArtistId, ar.Name, SUM(t.UnitPrice) AS profit
  FROM artists ar
  JOIN albums a ON ar.ArtistId = a.ArtistId
  JOIN tracks t ON a.AlbumId	= t.AlbumId
  JOIN genres g ON t.GenreId	= g.GenreId
  GROUP BY ar.ArtistId, ar.Name
  HAVING COUNT(distinct t.GenreId)>1
), pp AS
(
  SELECT ar.ArtistId, ar.Name, g.Name AS genre, SUM(t.UnitPrice) AS profit
  FROM artists ar
  JOIN albums a ON ar.ArtistId = a.ArtistId
  JOIN tracks t ON a.AlbumId = t.AlbumId
  JOIN genres g ON t.GenreId = g.GenreId
  GROUP BY ar.ArtistId, ar.Name, g.Name
)
SELECT pp.Name, pp.genre, (((pp.profit)/(p.profit))*100)  AS percent
FROM pp
JOIN p ON pp.ArtistId = p.ArtistId
WHERE pp.profit = (SELECT MAX(a.profit) FROM pp a WHERE pp.ArtistId = a.ArtistId)


 * sqlite:///store.db
Done.


Name,genre,percent
Antônio Carlos Jobim,Latin,57.70942155155734
Audioslave,Rock,42.39564428312156
Various Artists,Latin,55.773037167363135
Gilberto Gil,Soundtrack,46.82947729220222
Eric Clapton,Blues,72.312062937063
Faith No More,Alternative & Punk,71.61600415692392
Foo Fighters,Rock,78.97035468501856
Guns N' Roses,Rock,67.86613889888446
Iron Maiden,Metal,47.50791526585484
Jamiroquai,R&B/Soul,44.90504863362666
